#### Click on [Data](http://www.ats.ucla.edu/stat/data/binary.csv) to fetch the data for this notebook.

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

In [2]:
df=pd.read_csv('binary.csv',sep=',',names=['label','GPE','GRE','Rank'])
df.head()

,label,GPE,GRE,Rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [3]:
features=df.drop(columns='label')
labels=df['label']


features_with_dummies=pd.get_dummies(data=features,columns=['Rank'])
scaler=StandardScaler()

cols=['GPE','GRE']

features_with_dummies[cols]=scaler.fit_transform(features_with_dummies[cols])
features_with_dummies.head()

print(str(features_with_dummies['GPE'].mean())+'\t\t'+ str(features_with_dummies['GPE'].std()))

#np.random.seed(43)

test_indices=np.random.choice(features_with_dummies.index,size=int(len(features_with_dummies)*0.1), replace=False)

test_features,test_labels=features_with_dummies.iloc[test_indices], labels.iloc[test_indices]

train_indices=np.setdiff1d(np.array(features_with_dummies.index),np.array(test_indices))

train_features,train_labels=features_with_dummies.iloc[train_indices],labels.iloc[train_indices]

-4.061681546652096e-16		1.0012523486435176


In [4]:
n_rows,n_features=train_features.shape
print(n_features)

def sigmoid(x):
    return 1/(1+np.exp(-x))


6


In [5]:
learnrate=0.5
weights=np.random.normal(scale=1/np.sqrt(n_features), size=n_features)
bias=0


last_loss=None
epochs=1000


for e in range(epochs):
    del_w=np.zeros(weights.shape)
    for x,y in zip(train_features.values,train_labels):
        #print(x.shape, weights.shape)
    
        output=sigmoid(np.matmul(x,weights))

        error=y-output

        error_term = error*output*(1-output)

        del_w+=error_term*x
        weights+=learnrate*del_w/n_rows

    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(train_features, weights))
        loss = np.mean((out-y) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss
            
# Calculate accuracy on test data
tes_out = sigmoid(np.dot(test_features, weights))
predictions = tes_out > 0.5
#print(predictions)
accuracy = np.mean(predictions == test_labels)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  0.22571551776986504
Train loss:  0.19704585985276044
Train loss:  0.1970433162015207
Train loss:  0.19704331519357451
Train loss:  0.19704331519317483
Train loss:  0.1970433151931745
Train loss:  0.19704331519317447
Train loss:  0.19704331519317447
Train loss:  0.19704331519317447
Train loss:  0.19704331519317447
Prediction accuracy: 0.800


## Implementing Back Propagation

In [6]:
#paramters
learnrate=0.005
n_hidden=2
epochs=1000

#np.random.seed(21)


weights_input_hidden=np.random.normal(scale=1/np.sqrt(n_features),size=(n_features,n_hidden))
weights_hidden_output=np.random.normal(scale=1/np.sqrt(n_features),size=n_hidden)

last_loss=None

for e in range(epochs):
    for x,y in zip(train_features.values,train_labels.values):
        
        del_w_input_hidden=np.zeros(shape=weights_input_hidden.shape)
        del_w_hidden_output=np.zeros(shape=weights_hidden_output.shape)
        
        #forward pass
        hidden_layer_input=np.dot(x,weights_input_hidden)
        hidden_layer_output=sigmoid(hidden_layer_input)
        
        output_layer_in=np.dot(hidden_layer_output,weights_hidden_output)
        output=sigmoid(output_layer_in)
        
        #backward pass
        
        error=y-output
        error_term=error*output*(1-output)
        hidden_error_term=weights_hidden_output*error_term*hidden_layer_output*(1-hidden_layer_output)
        
        del_w_hidden_output+=error_term*hidden_layer_output
        del_w_input_hidden+=hidden_error_term*x[:,None]
        
    #weights update
    
    weights_hidden_output+=learnrate*del_w_hidden_output/n_rows
    weights_input_hidden+=learnrate*del_w_input_hidden/n_rows
    
    if e%(epochs/10)==0:
        hidden_out=sigmoid(np.dot(x,weights_input_hidden))
        out=sigmoid(np.dot(hidden_out,weights_hidden_output))
        
        loss=np.mean((y-out)**2)
        if last_loss and last_loss<loss:
            print("Train Loss: ", loss, "Warning-Loss Increasing")
        else:
            print("Train Loss: ", loss)
        
        last_loss=loss
        
#Accuracy
hidden=sigmoid(np.dot(test_features,weights_input_hidden))
test_out=sigmoid(np.dot(hidden,weights_hidden_output))
predictions=test_out>0.5
accuracy=np.mean(predictions==test_labels)
print("Prediction accuracy: {:.3f}".format(accuracy))            

Train Loss:  0.1740787289817364
Train Loss:  0.1740602723290662
Train Loss:  0.17404181769935584
Train Loss:  0.17402336509243663
Train Loss:  0.17400491450813985
Train Loss:  0.17398646594629688
Train Loss:  0.17396801940673892
Train Loss:  0.1739495748892974
Train Loss:  0.1739311323938038
Train Loss:  0.17391269192008932
Prediction accuracy: 0.775
